In [114]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import os.path

In [115]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/classroom.courses.readonly', # View your Google Classroom classes.
          'https://www.googleapis.com/auth/classroom.rosters', # Manage your Google Classroom class rosters.
          'https://www.googleapis.com/auth/classroom.profile.emails', # View the email addresses of people in your classes.
          'https://www.googleapis.com/auth/classroom.topics', #See, create, and edit topics in Google Classroom.
          'https://www.googleapis.com/auth/classroom.coursework.students', # Manage coursework and grades for students
          'https://www.googleapis.com/auth/classroom.courseworkmaterials', # See, edit, and create classwork materials in Google Classroom.
          'https://www.googleapis.com/auth/spreadsheets.readonly'] # See all your Google Sheets spreadsheets.


In [162]:
SCORE_ACADEMY_LINK = 'https://docs.google.com/spreadsheets/d/1cGJ0pn9k9gKCBnceWVwaL9D7BBDMNjLh8uPYlaBlJi8/edit?usp=sharing'
SCORE_ACADEMY_ID = SCORE_ACADEMY_LINK.split(sep='/')[-2]

In [163]:
NAMA_SHEET = "Academy: Batch 23"
STUDENT_RANGE = [f'{NAMA_SHEET}!B:B', f'{NAMA_SHEET}!D:D']

In [116]:
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())



In [164]:
try:
    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets().values().batchGet(spreadsheetId=SCORE_ACADEMY_ID,
                                                     ranges=STUDENT_RANGE).execute()
    values = sheet.get('valueRanges', [])
        
except HttpError as err:
    print(err)

In [195]:
import pandas as pd

nama = pd.DataFrame(values[0].get('values'))
email = pd.DataFrame(values[1].get('values'))

df = pd.concat([nama, email], axis=1)
df.columns = df.iloc[0]
df.drop(index=0, inplace=True)

In [198]:
df = df.reset_index()
df.drop(columns='index', inplace=True)

In [119]:
service = build('classroom', 'v1', credentials=creds)

In [120]:
results = service.courses().list(pageSize=5).execute()
courses = results.get('courses', [])

if not courses:
    print('No courses found.')

# Prints the names of the first 5 courses.
print('Courses:')
for course in courses:
    print(f"- {course['name']}")

Courses:
- Wizard Data Visualization
- Kementerian BUMN
- Automation
- Wizard Data Analytics
- Vulcan Machine Learning


In [121]:
course_input = input("Enter Google Classroom Class Name : ")
course_lowercase = course_input.lower()
course_id = None

for course in courses:
    if course_lowercase == course['name'].lower():
        course_id = course['id']
        break

if course_id == None:
    raise Exception(f"{course_input} course not found")

else:
    print(f'{course_input} found with ID {course_id}')

Wizard Data Visualization found with ID 621398536690


In [155]:
response = service.courses().students().list(courseId=course_id).execute()
students = response.get('students')

while response.get('nextPageToken'):
    response = service.courses().students().list(courseId=course_id, pageToken = response['nextPageToken']).execute()
    students += response.get('students')

In [171]:
import pandas as pd

dict = {'name': [],
        'email': []}

for student in students:
    dict['name'].append(student['profile']['name']['fullName'])
    dict['email'].append(student['profile']['emailAddress'])

submission = pd.DataFrame(dict)

In [184]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [236]:
email = []

for i in range(len(df)):
    poin = 0
    for j in range(len(submission)):
        if similar(df.loc[i, 'Name'], submission.loc[j, 'name']) >= poin :
            similiar_email = df.loc[i, 'Email']
            poin = similar(df.loc[i, 'Name'], submission.loc[j, 'name'])

    email.append(similiar_email)

df['email_classroom'] = email
df['same'] = df['Email'] == df['email_classroom']

df

,Name,Email,email_classroom,same
0,Bagus Danadyaksa Wijaya,danadyaksawijaya@gmail.com,danadyaksawijaya@gmail.com,True
1,Berliana Putri Nauli Harahap,berlianahar@gmail.com,berlianahar@gmail.com,True
2,Grace Angela Sutantya,angiegrace35@gmail.com,angiegrace35@gmail.com,True
3,Ihsan Azmi Rabbani,ihsan.azmi.rabbani@gmail.com,ihsan.azmi.rabbani@gmail.com,True
4,Luthfi Hadi Pratama,lhpratama2@gmail.com,lhpratama2@gmail.com,True
5,Melissa Rusli,melissa.offc@gmail.com,melissa.offc@gmail.com,True
6,Nadya Naftalia,nadyanaftalia@gmail.com,nadyanaftalia@gmail.com,True
7,Natalia Brenda E. Thadete,brenda.thadete@gmail.com,brenda.thadete@gmail.com,True
8,Ghozy Abror Aufan,ghozyaufan@gmail.com,ghozyaufan@gmail.com,True
9,Napoleon Charles Kambu,kambu.charles@gmail.com,kambu.charles@gmail.com,True


In [ ]:
df.to_csv("WizardDV_emails.csv")